# Fashion MNIST

## Validate Azure ML SDK installation and get version number for debugging purposes

In [ ]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace
Initialize a workspace object from persisted configuration.

In [ ]:
# Initialize Workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print("Resource group: ", ws.resource_group)
print("Location: ", ws.location)
print("Workspace name: ", ws.name)

## Create An Experiment
**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'fashion-mnist'
experiment = Experiment(workspace = ws, name = experiment_name)

## Create Linux DSVM as a compute target
**Note:** If creation fails with a message about Marketplace purchase eligibilty, go to portal.azure.com, start creating DSVM there, and select "Want to create programmatically" to enable programmatic creation. Once you've enabled it, you can exit without actually creating VM.

**Note:** By default SSH runs on port 22 and you don't need to specify it. But if for security reasons you switch to a different port (such as 5022), you can specify the port number in the provisioning configuration object.

In [ ]:
from azureml.core.compute import DsvmCompute
from azureml.core.compute_target import ComputeTargetException

compute_target_name = 'mydsvm'

try:
    dsvm_compute = DsvmCompute(workspace=ws, name=compute_target_name)
    print('found existing:', dsvm_compute.name)
except ComputeTargetException:
    print('creating new.')
    dsvm_config = DsvmCompute.provisioning_configuration(vm_size="Standard_D2_v2")
    dsvm_compute = DsvmCompute.create(ws, name=compute_target_name, provisioning_configuration=dsvm_config)
    dsvm_compute.wait_for_completion(show_output=True)

## Configure for using a Remote VM

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
run_config = RunConfiguration(framework="python")

# Set compute target to the Linux DSVM
run_config.target = dsvm_compute.name

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(
	conda_packages=['tensorflow', 'keras', 'matplotlib'])

## Submit the Experiment
Finally, run the training job on the ACI

In [ ]:
from azureml.core.script_run_config import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory='./scripts',
                                    script='train_Fashion_MNIST.py',
                                    run_config=run_config)

run = experiment.submit(script_run_config)
run.tag("Description","Remote VM trained Fashion MNIST model")

In [ ]:
run

In [ ]:
run.wait_for_completion(show_output=True)

## Show Metrics

In [ ]:
# get all metris logged in the run
run.get_metrics()
metrics = run.get_metrics()

import numpy as np
print('loss is {0:.2f}, and accuracy is {1:0.2f}'.format(
    metrics['loss'], 
    metrics['accuracy']
))

In [ ]:
# Plot data to see relationships in training and validation data
import numpy as np
import matplotlib.pyplot as plt
epoch_list = list(range(1, len(metrics['Training Accuracy']) + 1))  # values for x axis [1, 2, ..., # of epochs]
plt.plot(epoch_list, metrics['Training Accuracy'], epoch_list, metrics['Validation Accuracy'])
plt.legend(('Training Accuracy', 'Validation Accuracy'))
plt.show()

## Download Model

In [ ]:
# show all the files stored within the run record
run.get_file_names()

In [ ]:
import os
import shutil

run.download_file('outputs/saved_model.zip', output_file_path='../outputs')
shutil.unpack_archive('../outputs/saved_model.zip', './models/fashion_mnist/' + run.id.split('_')[1])
os.remove(path='../outputs/saved_model.zip')

## Clean up compute resource

In [ ]:
dsvm_compute.delete()